## Leverage

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
pd.options.display.float_format = '{:.2f}'.format
import plotly.graph_objects as go
from scipy.stats import norm

In [2]:
# Pull S&P 500 returns
ticker = 'SPY'
ret = yf.download(ticker, start='2010-01-01', progress=False)
ret.index = ret.index.to_period('D')
ret = ret["Adj Close"].resample("M").last()
ret = ret.pct_change()
ret.name = "ret"

d:\bbcx-investments-quarto\quarto\venv\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [3]:
# Leverage
leverage = 0.50  # 50% leverage
# leverage = 1.0  # 100% leverage

# Borrowing rate
r_b = 0.02
levered_ret = ret + leverage * (ret - r_b)

In [4]:
# Levered returns are riskier
fig = go.Figure()
trace0= go.Scatter(x=ret.index.to_timestamp("M"), y=ret, mode="lines", name=ticker)
trace1= go.Scatter(x=levered_ret.index.to_timestamp("M"), y=levered_ret, mode="lines", name=f'Levered: {leverage:0.1%}')
fig.add_trace(trace0)
fig.add_trace(trace1)
fig.layout.yaxis["title"] = "Monthly Return"
fig.update_yaxes(tickformat=".1%")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [5]:
print(f'Unlevered return SD is:\t {ret.std():.4f}')
print(f'Levered return SD is:\t {levered_ret.std():.4f}')

Unlevered return SD is:	 0.0427
Levered return SD is:	 0.0640


In [6]:
# Risk at different leverage
print(f'Levered return SD:')
for leverage in [0.0, 0.5, 1.0, 1.5, 2.0]:
    levered_ret = ret + leverage * (ret - r_b)
    print(f'  at leverage of {leverage:.2f}:\t {levered_ret.std():.4f}')

Levered return SD:
  at leverage of 0.00:	 0.0427
  at leverage of 0.50:	 0.0640
  at leverage of 1.00:	 0.0854
  at leverage of 1.50:	 0.1067
  at leverage of 2.00:	 0.1281


### Margin

In [7]:
# Initial margin
MARGIN = 50000
EQUITY = 100000
INIT_VALUE = MARGIN + EQUITY


pct_margin = EQUITY / INIT_VALUE
print(f'Initial percent margin is: {pct_margin: .1%}')

Initial percent margin is:  66.7%


In [8]:
# Percent margin after price move (ignoring interest exp accumulation)
RETURN = -0.10

new_asset_value = INIT_VALUE*(1+RETURN)
pct_margin = (new_asset_value-MARGIN)/new_asset_value
print(f'New percent margin is: {pct_margin: .1%}')

New percent margin is:  63.0%


In [9]:
# Simulate levered account
MAINTENANCE_MARGIN = 0.35
T = 20

# set up account
cols = ['assets', 'margin', 'equity', 'ret', 'pct_margin', 'margin call?']
indx = np.arange(0,T)
acct = pd.DataFrame(dtype=float,columns=cols, index=indx)

# Initialize accounts

acct.loc[0,'assets'] = INIT_VALUE
acct['margin']   = MARGIN  # assume interest expense will not accumulate in our time horizon
acct.loc[0,'equity'] = EQUITY

# simulate T returns (NOTE: remove random_state to make this truly random)
acct.ret = norm.rvs(loc=0.04, scale = 0.2, size=T, random_state=2)  
# acct.ret = norm.rvs(loc=0.04, scale = 0.2, size=T)  

# Calculate asset and equity values + percent margin each period
for t in acct.index[1:]:
    acct.loc[t,'assets'] = acct.loc[t-1,'assets'] * (1+acct.loc[t,'ret'])
acct.equity = acct.assets - acct.margin
acct.pct_margin = acct.equity/acct.assets

# Margin calls occur when pct_margin drops below maintenance margin
acct['margin call?'] = np.where(acct.pct_margin < MAINTENANCE_MARGIN, 1, 0)
acct


,assets,margin,equity,ret,pct_margin,margin call?
0,150000.00,50000,100000.00,-0.04,0.67,0
1,154312.00,50000,104312.00,0.03,0.68,0
2,94556.34,50000,44556.34,-0.39,0.47,0
3,129358.19,50000,79358.19,0.37,0.61,0
4,88133.40,50000,38133.40,-0.32,0.43,0
5,76821.53,50000,26821.53,-0.13,0.35,1
6,87620.81,50000,37620.81,0.14,0.43,0
7,69303.01,50000,19303.01,-0.21,0.28,1
8,57411.28,50000,7411.28,-0.17,0.13,1
9,49270.27,50000,-729.73,-0.14,-0.01,1


In [10]:
# Plot return time-series
fig = go.Figure()
trace= go.Scatter(x=acct.index, y=acct.pct_margin, hovertemplate="<br>Percent Margin: %{y:.1%}<br><extra></extra>")
fig.add_trace(trace)
# some formatting
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.layout.xaxis["title"] = "Time"
fig.layout.yaxis["title"] = "Percent Margin"
fig.add_hline(y=MAINTENANCE_MARGIN, line_width=4, line_dash="dash", line_color="black")
fig.add_annotation(x=3, y=MAINTENANCE_MARGIN*1.1,
            text="Maintenance margin: "+f'{MAINTENANCE_MARGIN:.1%}', showarrow=False)
fig.show()

### Repo cash flows

In [11]:
MV = 19576026.65
REPO_RATE = 0.06
HAIRCUT = 0.01

# Day 0 cash flow
cf0 = MV*(1-HAIRCUT)
print(f'Initial cash is: ${cf0:,.2f}')

# Day 1 cash flow
cf1 = cf0*(1+REPO_RATE/360)
print(f'Day 1 cash flow is: ${cf1:,.2f}')
print(f'Interest on cash loan is: ${cf1-cf0:,.2f}')

Initial cash is: $19,380,266.38
Day 1 cash flow is: $19,383,496.43
Interest on cash loan is: $3,230.04
